<a href="https://colab.research.google.com/github/ecordeiro/cefet/blob/main/busca_aleatoria.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [42]:
import numpy as np
import random
import altair as alt
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import math

import warnings
warnings.filterwarnings('ignore')

In [43]:
def plot_scatter_with_line_sea(populacao, x_title='X', y_title='Y', title='Scatter Plot with Connected Points', num_fronteira=0,name_chart=''):
    # Extract points from population

    points = [(ind.fitness[0], ind.fitness[1]) for ind in populacao if ind.domination_count == num_fronteira]

    if len(points)>0:

        # Unzip points into separate lists for x and y
        x_values, y_values = zip(*points)

        # Create scatter plot
        sns.scatterplot(x=x_values, y=y_values, label=title)

        # Create line plot
        sns.lineplot(x=x_values, y=y_values, linewidth=2, label='')

        # Add title and labels
        #plt.title(title)
        plt.xlabel(x_title)
        plt.ylabel(y_title)

        # Save the figure
        plt.savefig(name_chart)
        plt.close()

    # Display the plot
    #plt.show()

# Example usage:
# plot_scatter_with_line_sea(populacao)

In [44]:
class Individual:

    def __init__(self, chromosome_length, limites):
        # Inicializa um indivíduo com um cromossomo aleatório
        #self.fitness = None
        self.dominated_solutions = []  # Soluções dominadas por este indivíduo
        self.domination_count = 0  # Contagem de quantos indivíduos dominam este indivíduo
        self.limite_inferior, self.limite_superior = limites
        self.chromosome = [np.random.uniform(self.limite_inferior,self.limite_superior) for _ in range(chromosome_length)]
        self.crowding_distance = 0
        self.fitness = [self.f1(),self.f2()]
        self.fronteira = None

    def restricao_g(self, x):
        b = (np.sin(2 * np.pi * x) + 0.5) <= 0
        return 1 if b else 10

    def restricao_h(self, x):
        b = (np.cos(2 * np.pi * x) + 0.5) == 0
        return 1 if b else 10

    def f1(self):
        penalidade = 1
        n = len(self.chromosome)

        penalidade = penalidade * math.prod([self.restricao_g(x) for x in self.chromosome])
        penalidade = penalidade * math.prod([self.restricao_h(x) for x in self.chromosome])

        valor = sum([(elemento**2 - 4) - 10 * np.cos(2 * np.pi * (elemento**2 - 4)) for elemento in self.chromosome])
        return (10 * n + valor) * penalidade

    def f2(self):
        penalidade = 1
        n = len(self.chromosome)

        penalidade = penalidade * math.prod([self.restricao_g(x) for x in self.chromosome])
        penalidade = penalidade * math.prod([self.restricao_h(x) for x in self.chromosome])

        valor = sum([(elemento**2 - 10 * np.cos(2 * np.pi * elemento)) for elemento in self.chromosome])

        return (10 * n + valor) * penalidade


In [45]:
def dominancia(fitness1, fitness2):
    # Verifica se fitness1 domina fitness2
    return all(fitness1[i] <= fitness2[i] for i in range(len(fitness1))) and any(fitness1[i] < fitness2[i] for i in range(len(fitness1)))

def calcula_domininancia_populacao(p):
    # Zera as listas de soluções dominadas e contagens de dominação
    for ind in p:
        ind.dominated_solutions = []

    # Calcula as soluções dominadas e atualiza a contagem de dominação para cada indivíduo
    for i in range(len(p)):
        for j in range(i + 1, len(p)):
            if dominancia(p[i].fitness, p[j].fitness):
                p[j].dominated_solutions.append(i)
                p[i].domination_count += 1
            elif dominancia(p[j].fitness, p[i].fitness):
                p[i].dominated_solutions.append(j)
                p[j].domination_count += 1
    return p

def calcula_crowding_distance(p):
    # Calcula a distância de multidão de cada indivíduo
    for ind in p:
        ind.crowding_distance = 0

    num_objectives = len(p[0].fitness)
    # Calcula a distância de multidão para cada objetivo
    for m in range(num_objectives):
        p.sort(key=lambda x: x.fitness[m])
        p[0].crowding_distance = float('inf')
        p[-1].crowding_distance = float('inf')
        for i in range(1, len(p) - 1):
            p[i].crowding_distance += (p[i + 1].fitness[m] - p[i - 1].fitness[m])
    return p

def gerar_populacao(population_size,chromosome_length,limites):
    populacao = [Individual(chromosome_length, limites) for _ in range(population_size)]

    return populacao


In [46]:
tamanho_cromossomos = 10

df_full = pd.DataFrame()

for i in range(0, 12):

    qtd_individuos = 200
    limites = (-5.12, 5.12)

    populacao = gerar_populacao(qtd_individuos,tamanho_cromossomos,limites)

    num_geracoes = 1000
    qtd_individuos = 50
    limites = (-5.12, 5.12)

    for _ in range(num_geracoes):
        #Cria nova população
        nova_geracao = gerar_populacao(qtd_individuos,tamanho_cromossomos,limites)
        pop_total = populacao + nova_geracao
        populacao_total = calcula_domininancia_populacao(pop_total)
        populacao_total = calcula_crowding_distance(populacao_total)
        populacao_total.sort(key=lambda x: (x.domination_count, -x.crowding_distance))
        populacao = populacao_total[:len(populacao)]  # Seleciona os melhores indivíduos

    non_dominated_solutions = [ind for ind in populacao if ind.domination_count == 0]

    points = [(ind.fitness[0], ind.fitness[1]) for ind in populacao if ind.domination_count == 0]

    print(f"Solução {i} para {num_geracoes} gerações e n={tamanho_cromossomos}.", f"Pontos não dominados: {len(non_dominated_solutions)}")

    if len(non_dominated_solutions) > 0:
        df = pd.DataFrame({'Listas': f'Lista_{i}',f'Pontos': points})
        df_full = pd.concat([df_full,df])
        nome_chart = f'/content/drive/MyDrive/Cefet/busca_aleatoria/graficos/n{tamanho_cromossomos}/chart_n{tamanho_cromossomos}_numero_{i}.png'
        plot_scatter_with_line_sea(populacao,title='Fronteira 0',num_fronteira=0,name_chart=nome_chart)


Solução 0 para 1000 gerações e n=10. Pontos não dominados: 6
Solução 1 para 1000 gerações e n=10. Pontos não dominados: 2
Solução 2 para 1000 gerações e n=10. Pontos não dominados: 5
Solução 3 para 1000 gerações e n=10. Pontos não dominados: 2
Solução 4 para 1000 gerações e n=10. Pontos não dominados: 6
Solução 5 para 1000 gerações e n=10. Pontos não dominados: 4
Solução 6 para 1000 gerações e n=10. Pontos não dominados: 3
Solução 7 para 1000 gerações e n=10. Pontos não dominados: 4
Solução 8 para 1000 gerações e n=10. Pontos não dominados: 3
Solução 9 para 1000 gerações e n=10. Pontos não dominados: 2
Solução 10 para 1000 gerações e n=10. Pontos não dominados: 6
Solução 11 para 1000 gerações e n=10. Pontos não dominados: 5


In [47]:
# Salvar o DataFrame em um arquivo CSV
df_full.to_csv(f'/content/drive/MyDrive/Cefet/busca_aleatoria/listas_busca_aleatoria_n{tamanho_cromossomos}.csv', index=False)